# Postprocess an assembly {#assembly_example}

This example shows how to postprocess an assembly with multiple
composite parts. The assembly consists of a shell and solid composite
model. The `Composite Model <.CompositeModel>`{.interpreted-text
role="class"} class is used to access the data of the different parts.

::: note
::: title
Note
:::

When using a Workbench project, use the
`.composite_files_from_workbench_harmonic_analysis`{.interpreted-text
role="func"} method to obtain the input files.
:::


# Set up analysis

Setting up the analysis consists of loading Ansys libraries, connecting
to the DPF server, and retrieving the example files.

Load Ansys libraries.


In [ ]:
from ansys.dpf.composites.composite_model import CompositeModel
from ansys.dpf.composites.constants import FailureOutput
from ansys.dpf.composites.example_helper import get_continuous_fiber_example_files
from ansys.dpf.composites.failure_criteria import CombinedFailureCriterion, MaxStressCriterion
from ansys.dpf.composites.server_helpers import (
    connect_to_or_start_server,
    version_equal_or_later,
    version_older_than,
)

Start a DPF server and copy the example files into the current working
directory.


In [ ]:
server = connect_to_or_start_server()
composite_files_on_server = get_continuous_fiber_example_files(server, "assembly")

# Configure combined failure criterion

Configure the combined failure crition.


In [ ]:
combined_fc = CombinedFailureCriterion(
    name="failure of all materials",
    failure_criteria=[MaxStressCriterion()],
)

# Set up model

Set up the composite model.


In [ ]:
composite_model = CompositeModel(composite_files_on_server, server)

# Plot IRF

Plot the maximum IRF per (solid) element.


In [ ]:
output_all_elements = composite_model.evaluate_failure_criteria(combined_criterion=combined_fc)
irf_field = output_all_elements.get_field({"failure_label": FailureOutput.FAILURE_VALUE})
irf_field.plot()

# Plot IRF on the reference surface

Plot the maximum IRF on the reference surface


In [ ]:
if version_equal_or_later(server, "8.0"):
    irf_field = output_all_elements.get_field(
        {"failure_label": FailureOutput.FAILURE_VALUE_REF_SURFACE}
    )
    irf_field.plot()

# Get element information

In the assembly, two composite definitions exist: one with a \"shell\"
label and one with a \"solid\" label. For DPF Server versions earlier
than 7.0, the lay-up properties must be queried with the correct
composite definition label. The code following gets element information
for all layered elements. For DPF Server versions 7.0 and later, element
information can be retrieved directly.


In [ ]:
if version_older_than(server, "7.0"):
    element_infos = []
    for composite_label in composite_model.composite_definition_labels:
        for (
            element_id
        ) in composite_model.get_all_layered_element_ids_for_composite_definition_label(
            composite_label
        ):
            element_infos.append(composite_model.get_element_info(element_id, composite_label))

else:
    element_infos = []
    for element_id in composite_model.get_all_layered_element_ids():
        element_infos.append(composite_model.get_element_info(element_id))